In [89]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import time
import re
import datetime
from dateutil.relativedelta import relativedelta

In [94]:
month_dates = []
pages = [1,]
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
urls_list = []
titles = []
dates = []
articles = []
article_urls = []
categorys = []
press_companies = []
flag = 0

#한달전까지의 날짜 저장
# while 1:
#     temp = str(datetime.date.today() - datetime.timedelta(days = flag)).replace('-','')
#     one_month_ago = str(datetime.date.today() - relativedelta(months = 1)).replace('-','')
#     month_dates.append(temp)
#     flag += 1
#     if temp == one_month_ago:
#         break


# 최근 3일 날짜
while 1:
    temp = str(datetime.date.today() - datetime.timedelta(days = flag)).replace('-','')
    one_month_ago = str(datetime.date.today() - relativedelta(months = 1)).replace('-','')
    month_dates.append(temp)
    flag += 1
    if flag == 2:
        break

#마지막 페이지 리스트 추출
origin_url = 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=105&sid2=227'
origin_web = requests.get(origin_url, headers = headers).content
origin_source = BeautifulSoup(origin_web, 'html.parser')
for pageList in origin_source.find_all('a', {'class' : "nclicks(fls.page)"}):
    pages.append(pageList.get_text())

    
#날짜 및 페이지 수 => 뉴스링크 저장 
for date in month_dates:
    for page in pages:
        url = 'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=227&sid1=105&date={}&page={}'.format(date, page)
        web = requests.get(url, headers = headers).content
        source = BeautifulSoup(web, 'html.parser')
    
        for url in source.find_all('dt', {'class' : "photo"}):
            urls_list.append(url.find('a').attrs['href'])




for url in urls_list:
    try:
        web_news = requests.get(url, headers=headers).content
        source_news = BeautifulSoup(web_news, 'html.parser')

        # 1) 기사 제목
        title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()

        # 2) 기사 날짜
        date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

        # 3) 기사 본문
        article = source_news.find('article', {'id' : 'dic_area'}).get_text()
        article = article.replace("\n", "")
        article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
        article = article.replace("동영상 뉴스       ", "")
        article = article.replace("동영상 뉴스", "")
        article = article.strip()
        
        # 4) 기사 발행 언론사
        press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
        
        # 5) 카테코리
        category = source_news.find('em', {'class' : 'media_end_categorize_item'}).get_text()
        #print('Processing article : {}'.format(category))
        
        
        # 6) 리스트 저장
        titles.append(title)
        dates.append(date)
        articles.append(article)
        article_urls.append(url)
        press_companies.append(press_company)
        categorys.append(category)
    
    except:
        print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))
        
        
article_df = pd.DataFrame({'Date':dates,
                           'Category':categorys,
                           'Press':press_companies,
                           'Title':titles,  
                           'Document':articles,
                           'Link':article_urls})
# article_df = pd.DataFrame({'Date':dates,
#                            'Press':press_companies,
#                            'Title':titles,  
#                            'Document':articles,
#                            'Link':article_urls})

In [98]:
article_df['Date']

0      2023.09.24. 오후 6:57
1      2023.09.24. 오후 5:40
2      2023.09.24. 오후 5:39
3      2023.09.24. 오후 5:39
4      2023.09.24. 오후 5:33
              ...         
178    2023.09.22. 오후 3:07
179    2023.09.22. 오후 2:41
180    2023.09.22. 오후 2:38
181    2023.09.22. 오후 2:35
182    2023.09.22. 오후 2:30
Name: Date, Length: 183, dtype: object

In [96]:
article_df.to_csv('./three_days_news_scraping.csv')